<a href="https://colab.research.google.com/github/prakul/MongoDB-AI-Resources/blob/main/Langchain%3C%3EMongoDB_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install prerequisites dependencies


In [8]:
!pip install langchain langchain-mongodb langchain-openai pypdf pymongo openai python-dotenv tiktoken

# Setup the environment

In [17]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

load_dotenv(override=True)

# Add an environment file to the notebook root directory called .env with MONGO_URI="xxx" to load these envvars

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
MONGO_URI = os.environ["MONGO_URI"]
DB_NAME = "langchain-test"
COLLECTION_NAME = "test"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
MONGODB_COLLECTION = db[COLLECTION_NAME]

#DATA PREPARATION

 Create an Atlas search index via Atlas UI -> Search -> JSON Editor with the following definition
 ```
 {
   "fields":[
      {
         "type": "vector",
         "path": "embedding",
         "numDimensions": 1536,
         "similarity": "cosine"
      }
   ]
}
```

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2303.08774.pdf")
data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
docs = text_splitter.split_documents(data)

In [9]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-ada-002")


In [ ]:
from langchain_mongodb import MongoDBAtlasVectorSearch

# insert the documents in MongoDB Atlas Vector Search
vector_store = MongoDBAtlasVectorSearch.from_documents(
     documents=docs, embedding=embeddings, collection=MONGODB_COLLECTION, index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME
 )


#DATA QUERY

In [18]:
from langchain_mongodb import MongoDBAtlasVectorSearch

vector_search = MongoDBAtlasVectorSearch.from_connection_string(
    MONGO_URI,
    DB_NAME + "." + COLLECTION_NAME,
    OpenAIEmbeddings(disallowed_special=()),
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME
)


In [12]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
question = "How much better is GPT-4 in reducing hallucinations over GPT-3.5"
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [14]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vector_search.as_retriever())
qa_chain({"query": question})


{'query': 'How much better is GPT-4 in reducing hallucinations over GPT-3.5',
 'result': 'GPT-4 scores 19 percentage points higher than GPT-3.5 on internal evaluations at avoiding open-domain hallucinations.'}

-

-

-

-


-

-


-
















# APPENDIX

In [15]:
from langchain_mongodb.cache import MongoDBAtlasSemanticCache

semantic_cache = MongoDBAtlasSemanticCache(
    connection_string=MONGO_URI,
    embedding=embeddings,
    collection_name="semantic_cache",
    database_name=DB_NAME,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME
)


In [ ]:
semantic_cache.clear()

In [16]:
from langchain_core.globals import set_llm_cache
set_llm_cache(semantic_cache)